In [1]:
import numpy as np
import pandas as pd
import numba
from sympy import isprime
from math import sqrt

In [2]:
cities = pd.read_csv('../data/raw/cities.csv', index_col=['CityId'])

In [3]:
XY = np.stack((cities.X.astype(np.float32), cities.Y.astype(np.float32)), axis=1)
is_not_prime = np.array([not isprime(city_id) for city_id in cities.index], dtype=np.int32)

In [4]:
@numba.jit('f8(i8[:])', nopython=True, parallel=False)
def pure_score(path):
    '''Pure path score without penalties.'''
    dist = 0.0
    for i in numba.prange(path.shape[0] - 1):
        a, b = XY[path[i]], XY[path[i+1]]
        dx, dy = a[0] - b[0], a[1] - b[1]
        dist += sqrt(dx * dx + dy * dy)
    return dist

In [8]:
@numba.jit('f8(i8[:])', nopython=True, parallel=False)
def pure_score_v2(path):
    '''Pure path score without penalties.'''
    dist = 0.0
    for i in numba.prange(path.shape[0] - 1):
        a, b = XY[path[i]], XY[path[i+1]]
        dx, dy = a[0] - b[0], a[1] - b[1]
        dist += sqrt(dx * dx + dy * dy)
    return dist

In [5]:
path = np.concatenate([cities.index, [0]])

In [6]:
%timeit pure_score(path)

1.13 ms ± 9.48 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


In [7]:
pure_score(path)

443430860.1241331

In [10]:
pure_score_v2(path)

560957677.8028173